## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/DSL/[세션] 정규세션/[0830] Text Mining, Word Embedding/TextMining_WordEmbedding

/content/drive/MyDrive/DSL/[세션] 정규세션/[0830] Text Mining, Word Embedding/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [1,162 kB]
Hit:14 http://p

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['"아름다움과 복잡성 깜짝 놀랄만 해"',
 '뉴호라이즌호가 작년 7월부터 수집한 명왕성의 데이터를 모두 지구로 전송하는데 성공했다',
 '스페이스닷컴 등 외신은 나사 우주선 뉴호라이즌호가 명왕성을 저공비행하며 촬영한 데이터의 마지막 조각까지 전송했다고 28일 보도했다',
 '뉴호라이즌호는 지난 2006년 1월 지구에서 출발해 약 10년에 달하는 비행 끝에 작년 7월 명왕성 1만2500km미터까지 접근하는데 성공했다',
 '뉴호라이즌 호의 모습',
 '접근에 성공한 우주선은 저공 비행을 통해 약 50 기가바이트의 데이터를 수집했다',
 '수집된 데이터는 가장 중요도가 높은 순서대로 15개월에 걸쳐 지구로 전송됐다',
 '나사의 앨런 스턴 뉴호라이즌 연구 책임자는 "수집한 데이터를 통해 확인한 명왕성과 주변 위성의 아름다움과 복잡성은 우리를 깜짝 놀라게했다"며 "이제 지구로 보내진 데이터를 과학적인 관찰을 통해 이해하는 큰 작업이 남아있다"고 말했다',
 '나사는 명왕성 저공비행 데이터의 품질을 확인 한 후 결과물이 만족스러울 경우 뉴호라이즌호를 1억 6천km 떨어진 카이퍼 벨트트 소속의 천체 2014 MU69로 향하는 추가 임무를 진행할 예정이다',
 '- 기어 조작은 베트남 전쟁 당시 기관총 조작에 쓰였던 조종간이다',
 '- 배기량 3리터, 출력 210 마력의 엔진은 부서진 2002년형 아우디 V6 엔진에서 가져왔다',
 '엔진 가격은 450달러',
 '- 폰툰은 1988년형 선 트래커 보트에서 가져왔고, 강철로 보강했다',
 '지난해, 여기저기 부서진 1969년산 벨 OH-58 카이오와 헬리콥터의 동체가 경매에서 3,101달러에 팔렸다',
 '이 헬리콥터는 베트남 전쟁에도 참전했고, 연방 마약 수사 기동대에서도 사용되었으며, 이후 퇴역해 테네시 주 내시빌에서 해체되어 부품 조달용으로 사용되다가 홍수에 떠내려갔다',
 '하지만 이 헬리콥터는 폐기처분을 면하고 수륙양용 자동차로 제2의 삶을 살게 되었다',
 '그것은 이 헬리콥터의 새 주인인 제프 블로크의 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

- 데이터의 특성 상(다양한 주제, 외부에서 가져옴) stop word를 개별 지정하는 것보다 품사를 통해 stop words를 지정하는 것이 더욱 효율적이라 생각하여 품사를 통해 stop words를 지정하여 제거하는 방법을 사용했습니다.

In [8]:
okt = Okt()

In [9]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
    stop = ['Josa', 'Suffix', 'Punctuation', 'Foreign', 'Number']
    token = okt.pos(sentence)
    token = [i for i, j in token if j not in stop]
    return token

In [10]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [ ]:
corpus_tokenized[:5]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [25]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(hs=1, min_count=0)    # hierarchical softmax 사용 / 시각화를 위해 min_count=0

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4556


In [26]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(157598, 168430)

In [27]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('과학')

[('효과', 0.9963257312774658),
 ('되는', 0.9947048425674438),
 ('전자', 0.994189977645874),
 ('그', 0.9938929080963135),
 ('지난', 0.9928162097930908),
 ('하고', 0.9908886551856995),
 ('대학', 0.990757942199707),
 ('사람', 0.9901047348976135),
 ('로봇', 0.9897391200065613),
 ('현실', 0.9894472360610962)]

In [28]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, hs=1, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4556


In [29]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 32081


In [30]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(166051, 168430)

In [31]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('과학')

[('인문학', 0.6731842756271362),
 ('사회과학', 0.6213877201080322),
 ('교육학', 0.6194782853126526),
 ('의학', 0.6072076559066772),
 ('지리학', 0.5825418829917908),
 ('공학', 0.5821242928504944),
 ('사회학', 0.5802952647209167),
 ('인류학', 0.5748034715652466),
 ('심리학', 0.5738147497177124),
 ('첨단', 0.5628946423530579)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

- 사전 학습이 된 모델이 사전 학습 없는 모델보다 성능이 좋은 것 같습니다. most_similar()을 통해 제시어와 코사인 유사도가 높은 단어를 추출했을 때 사전 학습이 된 모델이 보다 구체적인 단어를 제시했습니다. 이러한 차이가 발생한 이유는 사전 학습 없이 학습한 모델은 800여 단어의 작은 데이터셋으로 학습한 반면 사전 학습이 된 모델은 몇 백 배 더 많은 단어로 학습했기 때문인 것으로 보입니다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [32]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4556

In [37]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_no_pretrained.wv[i])

In [38]:
pca = PCA().fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [44]:
len(vector)

4556

In [49]:
# 시각화해보세요!
# PCA


data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


In [51]:
# t-SNE
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [52]:
# t-SNE
tsne = TSNE(n_components = 2, 
            random_state = 0, 
            perplexity = 10, 
            learning_rate = 0.05, 
            init = 'random', 
            n_iter = 1000).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(len(vector))],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!